## Repliikide lausestamine test50

In [1]:
from estnltk import Text
import sqlite3
from estnltk.converters import json_to_text
from estnltk.converters import text_to_json
from estnltk_core.layer_operations import split_by_sentences

Tühi järjend repliikide hoiustamiseks:

In [2]:
corpus = []

Tühi järjend repliikide ID-de jaoks:

In [3]:
line_ids = []

Andmebaasi tabelist loetakse sisse repliikidele vastavad Text-objektid:

In [4]:
con = sqlite3.connect("media_data_2.db")

In [5]:
cur = con.cursor()

In [6]:
for row in cur.execute("SELECT ID, line FROM lines"):
    line_id = row[0]
    line = json_to_text(json_text=row[1])
    corpus.append(line)
    line_ids.append(line_id)

In [7]:
con.close()

In [8]:
len(corpus)

3586

In [14]:
test_text = corpus[100]
display(test_text)

Text(text='Ei see on selge aga mõtled imetleme neid asiseid huvitav mõttekäik.\nEt kuidas see Strandbergi su poolega punkris eks ole nagu sisse seal allakäike Untergang ja ka selle filmi seisis juustes on imerelvad välja vaid üks imerelv on see et tulge kõik eks ole siis teine imerelv oli lausemaad seda on kaelas seitse tuhat lille teeme.\nTeeme igaüks oma erakonna katsume saada et selle sellesse nende nende kaitse on sellised ained on seda seda ja seda loovat mõtet lihtsad maa see mis mis see tuleb ei saa aru ja siis seda me teame kõik nagu niisuguseid käiketest.\nSooloole selle otsusega nõus et need on naljakad ja kihtides ja noh ja nüüd on aeg väljadel Rein Marek Strandbergi peaks nõudma meid kedagi üllatada ta on looja ideega on luigel sulle hoitavate nii et need laariga tagamine käive oli teid välja väga suure huviga räägib väga kommenteerimine ideid.')

In [15]:
test_text.ner

Layer(name='ner', attributes=('nertag',), spans=SL[EnvelopingSpan(['Strandbergi'], [{'nertag': 'PER'}]),
EnvelopingSpan(['Untergang'], [{'nertag': 'PER'}]),
EnvelopingSpan(['Rein'], [{'nertag': 'ORG'}]),
EnvelopingSpan(['Marek', 'Strandbergi'], [{'nertag': 'PER'}])])

Repliike lausestan funktsiooni split_by_sentences abil. Küll aga ei lausesta see teksti, kus lauselõpumärgid puuduvad (näitena ülal esitatud repliik). Seetõttu jäävad sellised repliigid oma esialgsele kujule, kuigi need salvestatakse lihtsuse mõttes hiljem koos lausetega samasse tabelisse.

In [16]:
line_sentences = split_by_sentences(text=test_text,
                           layers_to_keep=list(test_text.layers),
                           trim_overlapping=True
                           )

In [17]:
# Lauselõpumärkideta repliik jääb samale kujule
for sentence in line_sentences:
    if len(sentence.ner) > 0:
        display(sentence)
        display(sentence.ner)
        break

Text(text='Et kuidas see Strandbergi su poolega punkris eks ole nagu sisse seal allakäike Untergang ja ka selle filmi seisis juustes on imerelvad välja vaid üks imerelv on see et tulge kõik eks ole siis teine imerelv oli lausemaad seda on kaelas seitse tuhat lille teeme.')

Layer(name='ner', attributes=('nertag',), spans=SL[EnvelopingSpan(['Strandbergi'], [{'nertag': 'PER'}]),
EnvelopingSpan(['Untergang'], [{'nertag': 'PER'}])])

In [21]:
con = sqlite3.connect("media_data_2.db")
cur = con.cursor()

Andmebaasi lisatakse uus tabel lausete jaoks:

In [22]:
cur.execute("CREATE TABLE sentences(ID INTEGER PRIMARY KEY, line_ID INTEGER, sentence TEXT, named_entity INTEGER)")

Loodud tabelisse lisatakse kõik laused, eraldi märge lisatakse veergu named_entity, kas lause sialdab mõnda NerTaggeri abil leitud nimeüksust või mitte:

In [23]:
for i in range(len(corpus)):
    line_sentences = split_by_sentences(text=corpus[i], 
                           layers_to_keep=list(corpus[i].layers),
                           trim_overlapping=True
                           )
    for sentence in line_sentences:
        sentence.meta['line_number'] = corpus[i].meta['line_number']
        sentence.meta['person'] = corpus[i].meta['person']
        sentence.meta['date'] = corpus[i].meta['date']
        sentence_json = text_to_json(sentence)
        if len(sentence.ner) > 0:
            cur.execute("""INSERT INTO sentences
                                    (line_ID, sentence, named_entity)
                                    VALUES (?, ?, ?);""", (line_ids[i], sentence_json, 1))
        else:
            cur.execute("""INSERT INTO sentences
                                    (line_ID, sentence, named_entity)
                                    VALUES (?, ?, ?);""", (line_ids[i], sentence_json, 0))
        con.commit()

In [24]:
con.close()